First Pass Descriptors for Protiens
---
made in collaboration with ai and maya talpai-vasinthascha


# How to Convert to .PDB File from .CIF

Most of the code below will use PBD files since the libraries I am pulling from most widely accept this file type. To convert the .CIF files to .PDB you can use a atom viewer like ChimeraX. Below are the steps you should take to convert file type.

**1.) Open the file in ChimeraX. Ensure to change the file name to the accurate name in your directory.***

    open <your .CIF file name>.cif

At this stage it is probaibly nice to get screen shots of your superimposed protien. Use matchmaker to align them so we ave some cool photos to add to slides and the prelimiary results.

**2.) Once your .CIF file is open save as a PDB file.** Please ensure that the file you convert is named either NONAD for the protein with no ligand docked and NAD for the protein with the liagnd attached.

**<font color="red">The file names must be exact to examples below!**

<u>For the .CIF file with NAD docked <u>

    save NAD.pdb


<u>For the .CIF file with the NAD *NOT* docked <u>


    save NONAD.pdb

**3.)Ensure your file are uploaded into this Colab notebook. On the side there is a bar with a file icon. Click the file Icon and upload the NONAD.pdb or NAD.pdb file**

#RMSD#

The Root Mean Square Difference calucaltes the average distance between each aligned *atom* in two superimposed protiens. This is done globally per atom, but more useful to look at per residue since looking for bigger picture strucutral differences.

This peice of code automates finding the RMSD between two protiens given their PBD files.

In [ ]:
!pip install MDAnalysis # https://www.mdanalysis.org/  #Run this once

In [ ]:
import warnings
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
from google.colab import files

warnings.filterwarnings("ignore", message="Reader has no dt information")
warnings.filterwarnings("ignore", message="Unit cell dimensions not found")
warnings.filterwarnings("ignore", message="Found no information for attr: 'formalcharges'")

NAD_pdb = "NAD.pdb" #if you didnt change your file name before, you can change it here
NONAD_pdb = "NONAD.pdb" #if you didnt change your file name before, you can change it here

def calc_rmsd(NAD, NONAD, selection="backbone"):
  #Performing alignment between two protein .CIF files
  nad = mda.Universe(NAD_pdb)
  nonad = mda.Universe(NONAD_pdb)
  nad_atoms = nad.select_atoms(selection)
  nonad_atoms = nonad.select_atoms(selection)
  align.AlignTraj(nad, nonad, select=selection).run() #align using atoms selected above (should be 747aa)

  # Calculate RMSD
  rmsd_value = rms.rmsd(nad_atoms.positions, nonad_atoms.positions)

  return rmsd_value

print("RMSD:", calc_rmsd(NAD_pdb, NONAD_pdb), "Å")

RMSD: 36.76266421385485 Å


# RMSD per Residue

The Root Mean Square Difference calucaltes the average distance between each aligned residue in two superimposed protiens. This is done locally per residue since looking for bigger picture strucutral differences.

This peice of code automates finding the RMSD between two protiens given their PBD files.

In [ ]:
import MDAnalysis as mda
import numpy as np
import pandas as pd

# Load the aligned file and the reference file
aligned = mda.Universe("rmsfit_NAD.pdb")
reference = mda.Universe("NAD.pdb")

# Select backbone atoms or another region of interest
selection = "all"
aligned_atoms = aligned.select_atoms(selection)
reference_atoms = reference.select_atoms(selection)

# Ensure selection sizes match
if len(aligned_atoms) != len(reference_atoms):
    raise ValueError("Selections do not match in size. Check your structures.")

# Calculate per-residue RMSD
residue_rmsd = []
for res_aligned, res_reference in zip(aligned_atoms.residues, reference_atoms.residues):
    # Calculate RMSD for all atoms in the current residue
    aligned_positions = res_aligned.atoms.positions
    reference_positions = res_reference.atoms.positions

    # Calculate RMSD for the residue
    rmsd_value = np.sqrt(np.mean((aligned_positions - reference_positions) ** 2))
    residue_rmsd.append((res_aligned.resname, res_aligned.resid, rmsd_value))

# Convert to DataFrame
residue_df = pd.DataFrame(residue_rmsd, columns=["Residue", "Residue Number", "RMSD (Å)"])

# You can filter results here with a high threshold
threshold = 0  # Adjust this threshold, I set to zero to get evrthing, but if we only want a high difference we can do here
significant_residues = residue_df[residue_df["RMSD (Å)"] > threshold]

significant_residues


/usr/local/lib/python3.10/dist-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


,Residue,Residue Number,RMSD (Å)
0,GLY,1,34.695850
1,SER,2,31.266632
2,ARG,3,34.941093
3,ASP,4,29.135435
4,ASN,5,28.414082
...,...,...,...
393,GLU,394,28.747169
394,ASP,395,29.456764
395,ASP,396,34.080837
396,VAL,397,36.264221


# Clustered and Ranked RMSD by residue

In [1]:
#This is looking at clusters looking at the RMSD per cluster

import MDAnalysis as mda
import numpy as np
import pandas as pd

aligned = mda.Universe("rmsfit_NAD.pdb")
reference = mda.Universe("NAD.pdb")
selection = "all"
aligned_atoms = aligned.select_atoms(selection)
reference_atoms = reference.select_atoms(selection)

# Calculate mean RMSD for each residue
residue_rmsd = []
for res_aligned, res_reference in zip(aligned_atoms.residues, reference_atoms.residues):
  # Calculate RMSD for all atoms in the current residue
  aligned_positions = res_aligned.atoms.positions
  reference_positions = res_reference.atoms.positions
  rmsd_value = np.sqrt(np.mean((aligned_positions - reference_positions) ** 2))
  residue_rmsd.append((res_aligned.resname, res_aligned.resid, rmsd_value))

# Convert to DataFrame becaue easier and rank all residues by RMSD and save to csv
residue_df = pd.DataFrame(residue_rmsd, columns=["Residue", "Residue Number", "RMSD (Å)"])
residue_df = residue_df.sort_values(by="Residue Number").reset_index(drop=True)
residue_df_sorted = residue_df.sort_values(by="RMSD (Å)", ascending=False)
residue_df_sorted.to_csv("ranked_residues_rmsd.csv", index=False)

#This is if you want to srt by RMSD, but this i just set to get them all
rmsd_threshold = residue_df["RMSD (Å)"].quantile(1)  # Adjust percentile as needed
high_rmsd_residues = residue_df[residue_df["RMSD (Å)"] >= rmsd_threshold]
clusters = []
current_cluster = []
residue_numbers = high_rmsd_residues["Residue Number"].tolist()
for i in range(len(residue_numbers)):
  if i == 0:
    current_cluster.append(residue_numbers[i])
  else:
    # Check if the current residue is consecutive or within a gap of 1 residue
    if residue_numbers[i] <= residue_numbers[i-1] + 1:
      current_cluster.append(residue_numbers[i])
    else:
      if len(current_cluster) > 1:
        clusters.append(current_cluster)
      current_cluster = [residue_numbers[i]]

# Append the last cluster if it has more than one residue
if len(current_cluster) > 1:
  clusters.append(current_cluster)

# Extract clustered residues
clustered_residues_list = []
for cluster in clusters:
  cluster_df = residue_df[residue_df["Residue Number"].isin(cluster)]
  clustered_residues_list.append(cluster_df)

# Combine all clusters into a single DataFrame
clustered_residues_df = pd.concat(clustered_residues_list, ignore_index=True)

# Save the clustered residues to a CSV file
clustered_residues_df.to_csv("high_rmsd_clusters.csv", index=False)
print("Clusters of high RMSD residues saved to 'high_rmsd_clusters.csv'")

# Optional display the clusters
for idx, cluster_df in enumerate(clustered_residues_list, start=1):
  print(f"\nCluster {idx}:")
  print(cluster_df)



ModuleNotFoundError: No module named 'MDAnalysis'

#pLDDT#

Predicted Local Distance Difference Test is a score produced for each residue in thhe protien indicating the model's confidence in the predicted position of that residue.

In [2]:
import MDAnalysis as mda
import numpy as np
import pandas as pd

# Load the two structures
pdb_file1 = "NAD.pdb"
pdb_file2 = "NONAD.pdb"

a = mda.Universe(pdb_file1)
b = mda.Universe(pdb_file2)

# Initialize lists to store per-residue pLDDT scores
residue_confidence_a = []
residue_confidence_b = []

# Extract per-residue pLDDT for NAD
for res_a in a.residues:
    # Get the B-factors for each atom in the residue, which represent pLDDT scores
    pLDDT_scores_a = res_a.atoms.tempfactors
    mean_pLDDT_a = np.mean(pLDDT_scores_a)

    # Append residue information and mean pLDDT
    residue_confidence_a.append([res_a.resname, res_a.resid, mean_pLDDT_a])

# Extract per-residue pLDDT for NONAD
for res_b in b.residues:
    # Get the B-factors for each atom in the residue, which represent pLDDT scores
    pLDDT_scores_b = res_b.atoms.tempfactors
    mean_pLDDT_b = np.mean(pLDDT_scores_b)

    # Append residue information and mean pLDDT
    residue_confidence_b.append([res_b.resname, res_b.resid, mean_pLDDT_b])

# Convert lists to DataFrames
df_a = pd.DataFrame(residue_confidence_a, columns=["Residue", "Residue Number", "Mean pLDDT (NAD)"])
df_b = pd.DataFrame(residue_confidence_b, columns=["Residue", "Residue Number", "Mean pLDDT (NONAD)"])

# Merge the two DataFrames on residue information to compare them side by side
df_combined = pd.merge(df_a, df_b, on=["Residue", "Residue Number"], how="inner")
print(df_combined)

#Save to CSV for easy analysis
df_combined.to_csv("per_residue_pLDDT_comparison.csv", index=False)



ModuleNotFoundError: No module named 'MDAnalysis'

# VK

In [ ]:
!pip install biopython

In [ ]:
# Import necessary libraries
!apt-get install dssp
import pandas as pd
from Bio.PDB import DSSP, MMCIFParser

# Paths to your CIF files
nad_cif_file = "/content/fold_5btr_with_nad_model_4.cif"
nonad_cif_file = "/content/fold_5btr_model_4.cif"

# Initialize parser
parser = MMCIFParser(QUIET=True)

# Function to parse DSSP and extract secondary structure per residue
def extract_secondary_structure_per_residue(cif_file, structure_id):
    # Parse the structure
    structure = parser.get_structure(structure_id, cif_file)

    # Run DSSP and get secondary structure data
    dssp = DSSP(structure[0], cif_file)

    # Extract information per residue
    residues = []
    secondary_structures = []

    for key in dssp.keys():
        residue_id = key[1]  # Residue number
        secondary_structure = dssp[key][2]  # Secondary structure code
        residues.append(residue_id)
        secondary_structures.append(secondary_structure)

    # Create a DataFrame for easier analysis
    df = pd.DataFrame({
        "Residue Number": residues,
        "Secondary Structure": secondary_structures
    })

    return df

# Get per-residue secondary structure for both NAD and NONAD structures
nad_secondary_structure_df = extract_secondary_structure_per_residue(nad_cif_file, "NAD")
nonad_secondary_structure_df = extract_secondary_structure_per_residue(nonad_cif_file, "NONAD")

# Display the first few entries of each DataFrame

nad_secondary_structure_df

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dssp is already the newest version (4.0.4-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


/usr/local/lib/python3.10/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Invalid mmCIF file use --verbose option to see errors

  warnings.warn(err)
/usr/local/lib/python3.10/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Invalid mmCIF file use --verbose option to see errors

  warnings.warn(err)


,Residue Number,Secondary Structure
0,"( , 1, )",-
1,"( , 2, )",-
2,"( , 3, )",-
3,"( , 4, )",-
4,"( , 5, )",-
...,...,...
392,"( , 393, )",G
393,"( , 394, )",G
394,"( , 395, )",T
395,"( , 396, )",T
